<center><h1 style="font-size:300%; font-family:cursive; background:Blue; padding:10px; color:white; border-radius: 30px 30px;"> How to deal with Toxic words?</h1></center>
<br>
<center><h1 style="font-size:200%; font-family:cursive;color:Blue; ">Jigsaw Rate Severity of Toxic Comments</h1></center>

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Libraries Import</b></h1></center>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import metrics
from tqdm.auto import tqdm
from sklearn.naive_bayes import MultinomialNB
from bs4 import BeautifulSoup
from tokenizers import (decoders,models,normalizers,pre_tokenizers,processors,trainers,Tokenizer)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings("ignore")

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Data Preparations </b></h1></center>

In [ ]:
TRAIN_DATA_PATH = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
VALID_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv"
TEST_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_valid = pd.read_csv(VALID_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)

In [ ]:
df_train.head()

In [ ]:
df_valid.head()

In [ ]:
df_test.head()

In [ ]:
df_train.shape,df_valid.shape,df_test.shape

In [ ]:
for col in ['toxic','severe_toxic','obscene','threat','insult','identity_hate']:
    print(f'------------------------{col}-----------------------')
    display(df_train.loc[df_train[col]==1,['comment_text',col]].sample(2))

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Feature Weights </b></h1></center>


In the previous competition the task was to perform multi-class classification. Text sample could be labeled with one or several categories or not labeled with any. Non-toxic comments represent the majority of text samples, while toxic comments are a minority class and extremely toxic comments are more rare than plain toxic.In this competition we have to score texts based on the level of toxicity. To get a toxicity score from the previous data we can use two approaches:

* Simply sum up all values in each row of the DataFrame. The toxicity score will vary between 0 and 6. However some unequally toxic samples could have the same score.
    
* Adjust the values in the DataFrame according to extremety of the category (for example, "toxic" and "severe toxic" should have different score) and then sum up per row values.


In [ ]:
cat_mtpl ={'obscene':0.16,'toxic':0.32,'threat':1.5,
          'insult':0.64,'severe_toxic':1.5,'identity_hate':1.5}


for category in cat_mtpl:
    df_train[category] = df_train[category]*cat_mtpl[category]
    
df_train['score'] = df_train.loc[:,'toxic':'identity_hate'].mean(axis = 1)
df_train['y'] = df_train['score']


min_len = (df_train['y']>0).sum()

df_y0_undersample = df_train[df_train['y'] == 0].sample(n=min_len, random_state=41)  # take non toxic comments

df_train_new = pd.concat([df_train[df_train['y'] > 0], df_y0_undersample])  # make new df

df_train_new.head()

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Text Cleaning</b></h1></center>

In [ ]:
'''def text_cleaning(text):
    template = re.compile(r'https?://\S+|www\.\S+')
    text = template.sub(r'',text)
    
    soup = BeautifulSoup(text,'lxml')
    only_text = soup.get_text()
    text = only_text
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r"[^a-zA-Z\d]"," ",text)
    text = re.sub(' +',' ',text)
    text = text.strip().lower()
    
    lemmatizer = WordNetLemmatizer()
    stop = stopwords.words('english')
    
    text  = ''.join([lemmatizer.lemmatize(word) for word in text.split( ' ')])
    
    text = ' '.join([word for word in text.split(' ') if word not in stop])
    
    return text '''

'''tqdm.pandas()
df_train_new['clean_text'] = df_train_new['comment_text'].progress_apply(text_cleaning)'''

'''df_test['text'] = df_test['text'].progress_apply(text_cleaning)'''

 <center><h1 style="font-size:150%; font-family:cursive; color:Red; border:solid; border-radius:10px 10px; padding:13px;"><b>For Now I have not used this text cleaning</b></h1></center>

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Tokenizer Train</b></h1></center>

In [ ]:
raw_t = Tokenizer(models.WordPiece(unk_token = "[UNK]"))
raw_t.normalizer = normalizers.BertNormalizer(lowercase = True)
raw_t.pre_tokenizer  = pre_tokenizers.BertPreTokenizer()

special_tokens = ["[UNK]","[PAD]","[CLS]","[SEP]","[MASK]"]

trainer = trainers.WordPieceTrainer(vocab_size =10000,
                                  special_tokens = special_tokens)

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_train_new[['comment_text']])

def get_training_corpus():
    for i in range(0,len(dataset),1000):
        yield dataset[i:i+1000]["comment_text"]

In [ ]:
raw_t.train_from_iterator(get_training_corpus(),trainer =trainer)

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Model Training</b></h1></center>

In [ ]:
from transformers import PreTrainedTokenizerFast

tokenizer =PreTrainedTokenizerFast(
tokenizer_object = raw_t,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",)

In [ ]:
def dummy_fun(doc):
    return doc
labels = df_train_new['y']
comment = df_train_new['comment_text']
tokenized_comments = tokenizer(comment.to_list())['input_ids']

vectorizer = TfidfVectorizer(
analyzer = 'word',
tokenizer = dummy_fun,
preprocessor = dummy_fun,
token_pattern =None)

com_tr = vectorizer.fit_transform(tokenized_comments)
com_tr

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Simple Ridge </b></h1></center>

In [ ]:
%%time
regressor =Ridge(random_state = 42,alpha =5)
regressor.fit(com_tr,labels)

In [ ]:
'''%%time
model = Ridge(alpha=0.5)
model.fit(com_tr,labels)'''

In [ ]:
'''%%time
l_model = Ridge(alpha=1.)
l_model.fit(com_tr,labels)'''

In [ ]:
'''%%time
s_model = Ridge(alpha=2.)
s_model.fit(com_tr,labels)'''

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Processing Validation Data</b></h1></center>

In [ ]:
less_toxic_comments = df_valid['less_toxic']
more_toxic_comments = df_valid['more_toxic']

less_toxic_comments = tokenizer(less_toxic_comments.to_list())['input_ids']
more_toxic_comments = tokenizer(more_toxic_comments.to_list())['input_ids']


less_toxic = vectorizer.transform(less_toxic_comments)
more_toxic = vectorizer.transform(more_toxic_comments)

y_pred_less = regressor.predict(less_toxic)
y_pred_more = regressor.predict(more_toxic)

In [ ]:
(y_pred_less<y_pred_more).mean()

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Final Prediction on Test Data</b></h1></center>

In [ ]:
texts = df_test['text']
texts = tokenizer(texts.to_list())['input_ids']
texts = vectorizer.transform(texts)

In [ ]:
df_test['prediction'] = regressor.predict(texts)
df_test = df_test[['comment_id','prediction']]

df_test['score'] = df_test['prediction']
df_test = df_test[['comment_id','score']]

 <center><h1 style="font-size:150%; font-family:solid; color:Blue; border:solid; border-radius:10px 10px; padding:13px;"><b>Submission File</b></h1></center>

In [ ]:
df_test.to_csv('./submission.csv', index=False)
df_test.head()

 <center><h1 style="font-size:150%; font-family:solid; color:DarkOrange; border:solid; border-radius:10px 10px; padding:13px;"><b> Updating...</b></h1></center>